## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [17]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 설명해주세요.")
# topic 이 입력

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [9]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리", "how": "간단하게"}
# how의 예시로는 영어로, 5살짜리 어린아이도 쉽게 이해할 수 있게 등등이 가능해짐

In [10]:
chain

PromptTemplate(input_variables=['how', 'topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 {how} 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000017EF1FEFFD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000017EF1FF4E50>, root_client=<openai.OpenAI object at 0x0000017EF1FEEB10>, root_async_client=<openai.AsyncOpenAI object at 0x0000017EF1FF41D0>, model_name='gpt-4.1-nano', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [11]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.



# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.



chain.invoke(input)
# invoke는 답변을 기다렸다가 완성이 되면 출력

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 이용해 모델이 문제를 해결하는 방법을 배우는 과정입니다. 일반적으로 다음과 같은 단계로 이루어집니다:\n\n1. **데이터 수집:** 모델이 학습할 수 있도록 많은 예제와 정보를 모읍니다.\n2. **모델 초기화:** 무작위로 시작하거나 기본값으로 모델을 만듭니다.\n3. **학습 과정:** 모델이 예제 데이터를 보고 정답과 비교하며, 예측이 얼마나 맞았는지 평가합니다.\n4. **오차 계산:** 예측과 실제 정답 간의 차이를 계산합니다.\n5. **가중치 조정:** 오차를 줄이기 위해 모델 내부의 파라미터(가중치)를 수정합니다. 이 과정은 반복적으로 이루어집니다.\n6. **반복:** 여러 번 데이터를 통해 학습하며, 모델이 점점 더 정확하게 예측할 수 있도록 개선됩니다.\n\n이 과정을 통해 인공지능은 주어진 문제에 대해 더 잘 해결할 수 있는 능력을 갖추게 됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 233, 'prompt_tokens': 24, 'total_tokens': 257, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_8fd43718b3', 'id': 'chatcmpl-BXhjHiKy1mnqgRuCeyyuLdFuWW4mX', 'service_tier': 'default', 'finish_reason': 'st

아래는 스트리밍을 출력하는 예시 입니다.

In [12]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력 -> 바로바로 실시간으로 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 이용해 모델이 문제를 해결하는 방법을 배우는 과정입니다. 일반적으로 다음과 같은 단계로 이루어집니다:

1. **데이터 수집:** 모델이 학습할 수 있도록 많은 예제와 정보를 모읍니다.
2. **모델 설계:** 문제를 해결할 수 있는 구조(예: 신경망)를 만듭니다.
3. **학습 과정:** 데이터를 모델에 입력하고, 모델이 예측한 결과와 실제 정답을 비교하여 차이(오차)를 계산합니다.
4. **오차 줄이기:** 오차를 최소화하도록 모델의 내부 파라미터(가중치)를 조정하는 과정을 반복합니다. 이를 통해 모델이 점점 더 정확하게 예측할 수 있게 됩니다.
5. **검증 및 테스트:** 학습이 끝난 후, 새로운 데이터로 모델이 잘 작동하는지 평가합니다.

이 과정을 통해 인공지능은 주어진 문제에 대해 더 잘 해결할 수 있도록 '학습'하게 되는 것입니다.

### 출력파서(Output Parser)


In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [14]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [18]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

AIMessage(content="인공지능 모델의 학습 원리는 데이터를 이용하여 모델이 특정 작업을 수행할 수 있도록 '지식'을 습득하는 과정입니다. 일반적으로 인공지능 모델, 특히 딥러닝 모델은 다음과 같은 과정을 통해 학습됩니다.\n\n1. 데이터 수집 및 준비  \n- 모델이 학습할 수 있도록 다양한 예제와 정보를 포함하는 데이터를 수집합니다.  \n- 데이터를 정제하고 전처리하여 모델이 이해하기 쉬운 형태로 만듭니다. 예를 들어, 텍스트 데이터를 숫자 벡터로 변환하거나 이미지 데이터를 정규화하는 작업이 포함됩니다.\n\n2. 모델 구조 설계  \n- 문제에 적합한 인공신경망(딥러닝 모델)의 구조를 설계합니다.  \n- 예를 들어, 분류 문제에는 CNN, RNN, Transformer 등 다양한 구조를 사용할 수 있습니다.\n\n3. 손실 함수 정의  \n- 모델이 얼마나 잘 수행하는지 평가하는 기준인 손실 함수를 정합니다.  \n- 예를 들어, 분류 문제에서는 교차 엔트로피 손실(cross-entropy loss)을 사용합니다.\n\n4. 학습(훈련) 과정  \n- 모델이 예측한 결과와 실제 정답 간의 차이(손실)를 계산합니다.  \n- 이 손실 값을 최소화하기 위해 최적화 알고리즘(주로 경사 하강법, 예: Adam, SGD)을 사용하여 모델의 가중치(weight)를 조정합니다.  \n- 이 과정은 여러 번(에포크, epoch) 반복되며, 데이터 전체를 여러 번 학습하면서 모델이 점점 더 정확하게 예측할 수 있도록 합니다.\n\n5. 검증 및 튜닝  \n- 별도의 검증 데이터를 사용하여 모델의 성능을 평가하고, 과적합(overfitting)을 방지하기 위해 하이퍼파라미터를 조정하거나 정규화 기법을 적용합니다.\n\n6. 테스트 및 배포  \n- 최종적으로 학습된 모델을 테스트 데이터로 평가하여 실제 환경에서의 성능을 검증하고, 이후 배포하여 실제 사용에 적합하게 만듭니다.\n\n요약하자면, 인공지능 모델의 학습은 데이터를 통해 모델이 입력과 출력 간의 

In [19]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 이용하여 모델이 특정 작업을 수행할 수 있도록 '지식'을 습득하는 과정입니다. 일반적으로 인공지능, 특히 딥러닝 모델의 학습 과정은 다음과 같은 단계로 이루어집니다.

1. 데이터 수집 및 준비
- 모델이 학습할 수 있도록 다양한 예제(데이터)를 수집합니다.
- 데이터는 정제(cleaning), 정규화(normalization), 라벨링(labeling) 등의 전처리 과정을 거칩니다.

2. 모델 구조 설계
- 문제에 적합한 인공신경망(딥러닝 모델)의 구조를 설계합니다.
- 예를 들어, 이미지 인식에는 CNN(합성곱 신경망), 자연어 처리에는 RNN 또는 Transformer 등을 사용합니다.

3. 손실 함수 정의
- 모델의 예측값과 실제값 간의 차이를 측정하는 지표인 손실(loss) 함수를 정합니다.
- 예를 들어, 분류 문제에서는 교차 엔트로피(cross-entropy)를 사용합니다.

4. 학습(훈련) 과정
- 모델이 데이터를 통해 예측을 수행하고, 손실 함수를 계산합니다.
- 이후, 역전파(backpropagation) 알고리즘과 최적화 알고리즘(예: 경사하강법, Adam 등)을 이용하여 모델의 가중치(weight)를 조정합니다.
- 이 과정을 여러 번 반복하면서 모델이 점점 더 정확하게 예측할 수 있도록 학습합니다.

5. 검증 및 평가
- 별도의 검증 데이터(validation set)를 사용하여 모델의 성능을 평가하고, 과적합(overfitting)을 방지하기 위한 조치를 취합니다.
- 최적의 하이퍼파라미터를 찾거나, 모델을 조정합니다.

6. 테스트 및 배포
- 최종적으로 테스트 데이터(test set)를 통해 모델의 성능을 검증하고, 실제 환경에 배포합니다.

요약하자면, 인공지능 모델의 학습은 데이터를 통해 모델이 입력과 출력 간의 관계를 파악하도록 하는 과정이며, 이를 위해 손실 함수를 최소화하는 방향으로 가중치를 조정하는 반복적인 최적화 과정이 핵심입니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [42]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [43]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [44]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
"Hello, I’d like to order, please. Could I see the menu?"  

- 한글 해석:  
"안녕하세요, 주문하고 싶어요. 메뉴를 볼 수 있을까요?"


In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [41]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
"Hello! I’d like to order a large pepperoni pizza, please."  
"Could you also add some extra cheese? And can I get it delivered to my address?"  

- 한글 해석:  
"안녕하세요! 큰 페퍼로니 피자를 주문하고 싶어요."  
"추가 치즈도 넣어 주세요. 그리고 배달이 될까요?"